<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/nilm_cde.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files

In [0]:
'''Sliding window function'''
def create_dataset(dataset, look_back=1):
    dataX = []
    for i in range(len(dataset)-look_back+1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
    return np.array(dataX)

In [4]:
#uploaded = files.upload()
'''Load data'''
#WHE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_WHE.csv']))['P']
WHE_data = pd.read_csv('Electricity_WHE.csv')['P']
uploaded = files.upload()
CDE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_CDE.csv']))['P']
#CDE_data = pd.read_csv('Electricity_CDE.csv')['P']
'''
CDE_data = pd.read_csv(base_dir+'Electricity_CDE.csv')['P']
CWE_data = pd.read_csv(base_dir+'Electricity_CWE.csv')['P']
DWE_data = pd.read_csv(base_dir+'Electricity_DWE.csv')['P']
FRE_data = pd.read_csv(base_dir+'Electricity_FRE.csv')['P']
HPE_data = pd.read_csv(base_dir+'Electricity_HPE.csv')['P']
HTE_data = pd.read_csv(base_dir+'Electricity_HTE.csv')['P']
'''

Saving Electricity_CDE.csv to Electricity_CDE.csv


"\nCDE_data = pd.read_csv(base_dir+'Electricity_CDE.csv')['P']\nCWE_data = pd.read_csv(base_dir+'Electricity_CWE.csv')['P']\nDWE_data = pd.read_csv(base_dir+'Electricity_DWE.csv')['P']\nFRE_data = pd.read_csv(base_dir+'Electricity_FRE.csv')['P']\nHPE_data = pd.read_csv(base_dir+'Electricity_HPE.csv')['P']\nHTE_data = pd.read_csv(base_dir+'Electricity_HTE.csv')['P']\n"

In [0]:
window_size=60

dataX_raw = create_dataset(WHE_data.as_matrix(), window_size)

#1-150,151-400,4501-5000,others

cde_Y_raw = CDE_data[window_size-1:].values.reshape([CDE_data.shape[0]-window_size+1,1])
dataX = np.concatenate([dataX_raw[0:472500,:],dataX_raw[475500:,:]],axis=0)
cde_Y = np.concatenate([cde_Y_raw[0:472500,:],cde_Y_raw[475500:,:]],axis=0)
categorized_cde_Y = np.zeros(cde_Y.shape)
categorized_cde_Y[[np.where((cde_Y>0)&(cde_Y<=150))[0]],:]=1
categorized_cde_Y[[np.where((cde_Y>150)&(cde_Y<=400))[0]],:]=2
categorized_cde_Y[[np.where((cde_Y>4500)&(cde_Y<=5000))[0]],:]=3
categorized_cde_Y[[np.where((cde_Y>400)&(cde_Y<=4500))[0]],:]=4
categorized_cde_Y[[np.where((cde_Y>5000))[0]],:]=4

cdeY=categorized_cde_Y[categorized_cde_Y>0].reshape(-1,1)
cdeX=dataX[[np.where(categorized_cde_Y>0)[0]],:][0]


encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
cdeY_1hot = encoder.fit_transform(cdeY)

X_train, X_test, y_train, y_test = train_test_split(cdeX, cdeY_1hot, test_size=0.1, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(X_train.astype(float))
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
batch_size=512
unit_num=128

In [0]:
'''RNN Model Definition'''
tf.reset_default_graph()

#define inputs
tf_x = tf.placeholder(tf.float32, [None, window_size,1])
tf_y = tf.placeholder(tf.int32, [None, 4])

lstm_cell =tf.contrib.rnn.BasicLSTMCell(num_units=unit_num)
outputs, (h_c, h_n) = tf.nn.dynamic_rnn(
    lstm_cell,                   # cell you have chosen
    tf_x,                      # input
    initial_state=None,         # the initial hidden state
    dtype=tf.float32,           # must given if set initial_state = None
    time_major=False,           # False: (batch, time step, input); True: (time step, batch, input)
)
l1 = tf.layers.dense(outputs[:, -1, :],64,activation=tf.nn.leaky_relu)
l2 = tf.layers.dense(l1,32,activation=tf.nn.leaky_relu)
l3 = tf.layers.dense(l2,16,activation=tf.nn.leaky_relu)
pred = tf.layers.dense(l3,4,activation=tf.nn.relu)

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.metrics.accuracy(          # return (acc, update_op), and create 2 local variables
    labels=tf.argmax(tf_y, axis=1), predictions=tf.argmax(pred, axis=1),)[1]

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
sess = tf.Session()
sess.run(init_op)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("logss/", sess.graph) # tensorflow >=0.12
saver = tf.train.Saver()

In [0]:
!rm -r logss

In [0]:
batch_num = len(X_train)//batch_size
flag = False
for j in range(0,5000):
    print('###epoch: '+str(j)+'###')
    for i in range(0,batch_num+1):
        if(i!=batch_num):
            batch_X = X_train[i*batch_size:(i+1)*batch_size,].reshape([batch_size,window_size,1])
            batch_Y = y_train[i*batch_size:(i+1)*batch_size,]
        else: 
            batch_X = X_train[-batch_size:].reshape([batch_size,window_size,1])
            batch_Y = y_train[-batch_size:,]
        sess.run(train_op,{tf_x:batch_X,tf_y:batch_Y})
        cost_ = sess.run(loss, {tf_x: batch_X, tf_y:batch_Y})
        acc_train = sess.run(accuracy,{tf_x: batch_X, tf_y:batch_Y})
        acc_test = sess.run(accuracy,feed_dict={tf_x: X_test.reshape([X_test.shape[0],window_size,1]), tf_y:y_test})
        if i%10 == 0:
            rs = sess.run(merged,feed_dict={tf_x: batch_X, tf_y:batch_Y})
            pre = sess.run(pred,feed_dict={tf_x: batch_X, tf_y:batch_Y})
            writer.add_summary(rs, i+j*batch_num)
            y_lables_argmax = tf.argmax(tf_y,axis=1)  
            y_pred_argmax = tf.argmax(pre,axis=1)
            print('###Batch: '+str(i)+': train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
            print('Test Acc=%.2f'% acc_test)
            if acc_train>=0.97:
              print(j)
              save_path = saver.save(sess, "my_net/save_net_rnn.ckpt")
              flag = True
              break
    if(flag==True):
        print(flag)
        break
sess.close()


###epoch: 0###
###Batch: 0: train loss= 1.3810, Acc=0.17
Test Acc=0.19
###Batch: 10: train loss= 1.3365, Acc=0.26
Test Acc=0.26
###Batch: 20: train loss= 1.2694, Acc=0.28
Test Acc=0.28
###Batch: 30: train loss= 1.2246, Acc=0.31
Test Acc=0.31
###epoch: 1###
###Batch: 0: train loss= 1.1524, Acc=0.33
Test Acc=0.33
###Batch: 10: train loss= 1.1178, Acc=0.34
Test Acc=0.34
###Batch: 20: train loss= 1.0776, Acc=0.35
Test Acc=0.35
###Batch: 30: train loss= 1.0479, Acc=0.36
Test Acc=0.36
###epoch: 2###
###Batch: 0: train loss= 0.9768, Acc=0.36
Test Acc=0.36
###Batch: 10: train loss= 0.9875, Acc=0.37
Test Acc=0.37
###Batch: 20: train loss= 0.9967, Acc=0.37
Test Acc=0.37
###Batch: 30: train loss= 0.9976, Acc=0.37
Test Acc=0.37
###epoch: 3###
###Batch: 0: train loss= 0.9395, Acc=0.38
Test Acc=0.38
###Batch: 10: train loss= 0.8926, Acc=0.38
Test Acc=0.38


In [0]:
print(1)